In [1]:
%cd /Users/drbh/Documents/00000/PrebuildLambda/guess_speaker

/Users/drbh/Documents/00000/PrebuildLambda/guess_speaker


In [2]:
!ls -lah

total 15928
drwxr-xr-x   8 drbh  staff   272B Jan  3 15:40 .
drwxr-xr-x  17 drbh  staff   578B Jan  3 15:40 ..
-rw-r--r--@  1 drbh  staff   8.0K Jan  3 15:40 .DS_Store
-rw-r--r--   1 drbh  staff   1.0K Dec 29 00:31 audios.py
drwxr-xr-x@ 12 drbh  staff   408B Dec 25 16:25 feature
drwxr-xr-x@ 14 drbh  staff   476B Dec 26 18:27 filters
drwxr-xr-x@ 27 drbh  staff   918B Dec 28 17:34 gui
-rw-r--r--   1 drbh  staff   7.8M Jan  2 16:51 model.out


In [3]:
from collections import defaultdict
from scipy.io import wavfile
import numpy as np
import cPickle as pickle
import traceback as tb
import itertools
import glob
import time
import os
import sys
import operator

# add to run in ipython
sys.path.append('./gui/')

from interface import ModelInterface
from utils import read_wav
from filters.silence import remove_silence
from feature import mix_feature

No module named ap


In [4]:
execfile("./audios.py")

In [12]:
def see_model(mods):
    for count, (key, value) in enumerate(mods.features.iteritems(), 1):
        print count, key, len(value)

def load_local_model():
    mods = ModelInterface()
    mods = mods.load("./model.out")
    print see_model(mods)
    return mods

In [13]:
mods = load_local_model()

1 Kathy 2775
2 Luke 2775
3 David 3515
4 Nick 2590
5 Joe 2405
6 Lauren 2405
None


In [14]:

input_files = "/tmp/temp.wav"

for f in glob.glob(os.path.expanduser(input_files)):
    fs, signal = read_wav(f)
    label = mods.predict(fs, signal)
    print max(label, key=operator.itemgetter(1))

('David', -0.19238028712389044)


In [6]:
import json
import urllib
s3 = boto3.client('s3')
bucket = event['Records'][0]['s3']['bucket']['name']
key = urllib.unquote_plus(event['Records'][0]['s3']['object']['key'].encode('utf8'))
print "Getting file:", key, "from bucket", bucket
s3.download_file(bucket, key, '/tmp/temp.wav')
os.path.exists("/tmp/temp.wav")
print "File saved to: /tmp/temp.wav"

Getting file: 03_my_name.wav from bucket asu.datascience
File saved to: /tmp/temp.wav


In [5]:
import boto3
event = {
  "Records": [
    {
      "eventVersion": "2.0",
      "eventTime": "1970-01-01T00:00:00.000Z",
      "requestParameters": {
        "sourceIPAddress": "127.0.0.1"
      },
      "s3": {
        "configurationId": "testConfigRule",
        "object": {
          "key": "03_my_name.wav"
        },
        "bucket": {
          "name": "asu.datascience"
        },
        "s3SchemaVersion": "1.0"
      },
      "eventSource": "aws:s3"
    }
  ]
}
